<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load data</a></span><ul class="toc-item"><li><span><a href="#Load-network" data-toc-modified-id="Load-network-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load network</a></span></li><li><span><a href="#Load-cases" data-toc-modified-id="Load-cases-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Load cases</a></span></li><li><span><a href="#Run" data-toc-modified-id="Run-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Run</a></span></li></ul></li></ul></div>

In [ ]:
using Pkg; Pkg.activate()

using DataFrames, CSV
using Convex, ECOS
using Distributions, Random  
using LightGraphs  
using Plots

using SparseArrays
using Base.Iterators: product

using Revise
using CarbonNetworks

ECOS_QUIET = () -> ECOS.Optimizer(verbose=false)

In [ ]:
using Plots

In [ ]:
plotly()

# Load data

## Load network

In [ ]:
DATAPATH = ENV["CARBON_NETWORKS_DATA"]

In [ ]:
A, gmax, pmax, f, G, nodes = parse_network_data(DATAPATH; num_generators=1)
n = length(gmax)
m = length(pmax)

# Remove infinite capacities
pmax[pmax .== Inf] .= 3*maximum(gmax);

In [ ]:
normalization = mean(gmax)

gmax /= normalization
pmax /= normalization

@show n, m;

agg_nodes, B = create_generation_map(nodes);

In [ ]:
#removing constraints
max_val = 1.5*maximum(gmax)
gmax .= max_val
pmax .= 10*max_val;

## Load cases

In [ ]:
file_names = joinpath.(joinpath(DATAPATH, "TRAIN"), readdir(joinpath(DATAPATH, "TRAIN")))

cases = []
for file in file_names   
    d, g, case = load_case(file, agg_nodes, B, nodes)
    d = d/normalization
    g = g/normalization
    push!(cases, (d=d, g=g, case=case))
end

train_cases = cases[1:500]
test_cases = cases[501:end];

# train_cases = cases[1:1];
# test_cases = cases[2:2];

## Run

In [ ]:
Random.seed!(1);

In [ ]:
reinit=true

In [ ]:
# Algorithm parameters
step_size = .1
max_iter = 300
batch_size = 1
test_batch_size = 1
α = 1000

# Initialize estimated costs
if reinit
    f̂ = rand(Exponential(5), n) .+ 2
end

train_loss_hist, test_loss_hist, grad_hist, f̂ = CarbonNetworks.run_gradient_descent(
    f̂, train_cases, test_cases, pmax, gmax, A, B, 
    step_size, max_iter, batch_size, test_batch_size, 
    α
    );

In [ ]:
theme(:default, lw=4, label=nothing)

plot(
    plot(train_loss_hist, ylabel="train"),
    plot(test_loss_hist, ylabel="test"),
    layout=(2, 1), ylims=(0, 2)
)

In [ ]:
grad_hist_norm = []
for g in grad_hist
    push!(grad_hist_norm, norm(g))
end

In [ ]:
plot(grad_hist_norm, ylabel="gradien norm")

In [ ]:
case = train_cases[1];

ĝ, _, _ = CarbonNetworks.solvePMP(f̂, case, pmax, gmax, A);

In [ ]:
bar(case.g*normalization)

In [ ]:
rel_error = abs.(B*ĝ-case.g)./case.g*100;

In [ ]:
finite_idx = case.g.!=0;
case.g[finite_idx];

In [ ]:
histogram(case.g[finite_idx])

In [ ]:
histogram(rel_error[finite_idx])

In [ ]:
function logrange(x1, x2, n)
    v = [10^y for y in range(x1, x2, length=n)]
    return v
end

In [ ]:
v = logrange(-1, 6, 20);

In [ ]:
histogram(rel_error[finite_idx], xaxis=(:log10, (.1,Inf)), bins=v, xlabel="Relative Error [%]", ylabel="Counts")

In [ ]:
bar(rel_error, xlabel="generator #", ylabel="rel error [%]")

In [ ]:
n_gen_lower = []
for vi in v
    push!(n_gen_lower, sum(rel_error[finite_idx].≤vi))
end

n_gen_lower = n_gen_lower./length(rel_error[finite_idx])*100;

In [ ]:
plot(v, n_gen_lower, 
    xlabel="Relative error [%]", ylabel="CDF of relative errors over generators [%]", 
    xaxis=(:log10, (.1, Inf))
)